In [3]:
# Google Colab Setup
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
pip install pymdptoolbox

In [5]:
# Imports
import numpy as np
import pandas as pd
import os
from enum import Enum, auto
from collections import namedtuple
from itertools import product
import mdptoolbox
from copy import deepcopy

In [6]:
os.chdir("/content/gdrive/MyDrive/data" )

In [7]:
# Paths
old_training_file = 'Copy of SAKI Exercise 3 warehousetraining2x2.txt'
old_test_file = 'Copy of SAKI Exercise 3 warehouseorder2x2.txt'
new_training_file = 'warehousetraining.txt'
new_test_file = 'warehouseorder.txt'
TRAINING_DATASET_PATH = os.getcwd() + '/' + old_training_file
TEST_DATASET_PATH = os.getcwd() + '/' + old_test_file

In [8]:
# Constants
WAREHOUSE_SIZE = 4
WAREHOUSE_LAYOUT = (2,2)

In [9]:
Action = namedtuple('Action', ['operation_type','item_type'])

In [10]:
class OperationType(Enum):
    STORE = auto()
    RESTORE = auto()
    
    @staticmethod
    def from_str(value):
        if value.upper() == 'STORE':
            return OperationType.STORE
        elif value.upper() == 'RESTORE':
            return OperationType.RESTORE
        else:
            raise NotImplementedError

In [11]:
class ItemType(Enum):
    WHITE = auto()
    BLUE = auto()
    RED = auto()
    
    @staticmethod
    def from_str(value):
        if value.upper() == 'WHITE':
            return ItemType.WHITE
        elif value.upper() == 'BLUE':
            return ItemType.BLUE
        elif value.upper() == 'RED':
            return ItemType.RED
        else:
            raise NotImplementedError
    
    @staticmethod
    def list():
        return list(ItemType)

In [12]:
class WarehouseDataSet:
    
    def __init__(self, path=TRAINING_DATASET_PATH):
        self.path = path
        self.dataset = pd.read_csv(path, sep='\t', names=['OperationType', 'ItemType'])
        self.rel_freq = self._create_rel_freq_dict(self.dataset)
        self.size = self.dataset.shape[0]
    
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index < self.dataset.shape[0]:
          entry = self.dataset.iloc[self.index].tolist()
          self.index += 1
          operation_type = OperationType.from_str(entry[0])
          item_type = ItemType.from_str(entry[1])
          return Action(operation_type, item_type)
        else:
          raise StopIteration


    def _create_rel_freq_dict(self, dataset: pd.DataFrame):
        rel_freq_series = (dataset.value_counts() / dataset.shape[0])
        tmp_dict = rel_freq_series.to_dict()
        rel_freq_dict = {}
        for (operation, item), rel_freq in tmp_dict.items():
            operation_type = OperationType.from_str(operation)
            item_type = ItemType.from_str(item) 
            a = Action(operation_type, item_type)
            rel_freq_dict[a] = rel_freq
            #rel_freq_dict[(operation_type, item_type)] = rel_freq
        return rel_freq_dict

    def get_relative_frequency_for(self, action: (OperationType, ItemType)):
        return self.rel_freq.get(action, 0)

    def get_relative_frequencies(self):
        return self.rel_freq.copy()

In [14]:
class Warehouse:
     
    @staticmethod
    def is_applicable_action(position, s, s_prime):
        action = s[1]
        cell_content_s = s[0][position]
        cell_content_s_prime = s_prime[0][position]
        is_applicable = False
        if action.operation_type == OperationType.STORE:
            is_applicable = cell_content_s is None and cell_content_s_prime == action.item_type
        elif action.operation_type == OperationType.RESTORE:
            is_applicable = cell_content_s == action.item_type and cell_content_s_prime is None
        else:
            raise ValueError()
        return is_applicable
    

    @staticmethod
    def apply_at_to(action, position, warehouse_state):
        warehouse_state_prime = deepcopy(warehouse_state)
        if action.operation_type == OperationType.STORE and warehouse_state[position] is None:
            warehouse_state_prime[position] = action.item_type
        elif action.operation_type == OperationType.RESTORE and action.item_type == warehouse_state[position] :
            warehouse_state_prime[position] = None
        else:
          print(warehouse_state, action)
          raise ValueError
        return warehouse_state_prime


    @staticmethod
    def calculate_distance(position, layout):
        # Since the robot can not go diagonally
        # it makes sense to use the manhattan distance + 1 for the outside position
        manhattan_distance = sum(np.unravel_index(position, layout))
        return 1 + manhattan_distance



In [15]:
def create_transition_probability_matrix(dataset, state_space):
    number_of_states = len(state_space)
    tpm = np.zeros((WAREHOUSE_SIZE, number_of_states, number_of_states), dtype=np.float16)
    for position in range(WAREHOUSE_SIZE): 
        for x, s in enumerate(state_space):
            for y, s_prime in enumerate(state_space):
                if Warehouse.is_applicable_action(position, s, s_prime):
                    tpm[position, x, y] = dataset.get_relative_frequency_for(s[1])
            # handle dead states
            if tpm[position, x].sum() == 0:
                tpm[position, x, x] = 1.0
        tpm[position] = tpm[position] / tpm[position].sum(axis=1)[:, None]    
    return tpm

In [16]:
class ActionSpace:

    def __init__(self, operation_enum, item_type_enum):
        self.operation_enum = operation_enum
        self.item_type_enum = item_type_enum
    
    def get(self):
        action_space = []
        for o_type, i_type in product(list(self.operation_enum), list(self.item_type_enum)):
            action_space.append(Action(o_type, i_type))
        return action_space

In [17]:
class StateSpace:
    
    def __init__(self, operation_enum, item_type_enum, number_of_cells, action_space):
        self.number_of_cells = number_of_cells
        self.operation_enum = operation_enum
        self.item_type_enum = item_type_enum
        self.possible_cell_states = [None] + list(item_type_enum)
        self.warehouse_states = product(self.possible_cell_states, repeat=self.number_of_cells)
        self.action_space = action_space
        
    def get(self):
        return list(product(self.warehouse_states, self.action_space))
        

In [18]:
def create_state_space():
    from itertools import product
    states_space = []
    possible_cell_states = [None, ItemType.RED, ItemType.WHITE, ItemType.BLUE]
    warehouse_state_space = product(possible_cell_states, repeat=4)
    possible_actions = []
    t = product(list(OperationType), list(ItemType))
    for operation_type, item_type in t:
        possible_actions.append(Action(operation_type, item_type))
    return list(product(warehouse_state_space, possible_actions))

In [19]:
class RewardStructure:
    
    def __init__(self, reward_for_dead_ends, position_rewards, weights):
        self.reward_for_dead_ends = reward_for_dead_ends
        self.position_reward_mapping = {}
        for pos, reward in position_rewards:
            self.position_reward_mapping[pos] = reward
        self.weights = weights

    def calculate_reward(self, position, s):
        cell_content = s[0][position]
        action = s[1]
        reward = self.reward_for_dead_ends
        weight = self.get_weight_factor_for_action(action)
        if action.operation_type == OperationType.STORE:
            if cell_content is None:
                reward = self.position_reward_mapping.get(position) * weight
        elif action.operation_type == OperationType.RESTORE:
            if cell_content == action.item_type:
                reward = self.position_reward_mapping.get(position) * weight
        else:
            raise ValueError
        return reward
        
    def create_reward_matrix(self, dataset, state_space):
        number_of_states = len(state_space)
        rm = np.zeros((WAREHOUSE_SIZE, number_of_states), dtype=np.float16)
        for position in range(WAREHOUSE_SIZE):
            for x, s in enumerate(state_space):
                rm[position, x] = self.calculate_reward(position, s) 
        return rm.T
            
    def get_weight_factor_for_action(self, action):
        return self.weights.get(action, 1)


### Creating the transition probabilty matrix

In [20]:
action_space = ActionSpace(OperationType, ItemType).get()

In [21]:
state_space = StateSpace(OperationType, ItemType, WAREHOUSE_SIZE, action_space).get()

In [21]:
train_dataset = WarehouseDataSet(TRAINING_DATASET_PATH)
test_dataset = WarehouseDataSet(TEST_DATASET_PATH)

In [22]:
transition_probability_matrix = create_transition_probability_matrix(train_dataset, state_space)

In [23]:
position_rewards = []
for pos in range(WAREHOUSE_SIZE):
    reward = - Warehouse.calculate_distance(pos, WAREHOUSE_LAYOUT)
    position_rewards.append((pos, reward))

In [24]:
weights = {}
for action, rel_freq in train_dataset.get_relative_frequencies().items(): 
    weights[action] = 1 + rel_freq

In [39]:
reward_structure = RewardStructure(-10, position_rewards, weights)

In [26]:
reward_matrix = rw.create_reward_matrix(train_dataset, state_space)

### MDP

In [27]:
DISCOUNT_FACTOR = 0.9
ITERATIONS = 1000
policy_iteration_result = mdptoolbox.mdp.PolicyIteration(
    transition_probability_matrix,
    reward_matrix,
    DISCOUNT_FACTOR, 
    max_iter=ITERATIONS
)
value_iteration_result = mdptoolbox.mdp.ValueIteration(
    transition_probability_matrix,
    reward_matrix, 
    DISCOUNT_FACTOR, 
    max_iter=ITERATIONS
)

policy_iteration_result.run()
value_iteration_result.run()

print('Policy-Iteration Algorithm:')
print(policy_iteration_result.policy)
print(policy_iteration_result.V)
print(policy_iteration_result.iter)

print('Value-Iteration Algorithm:')
print(value_iteration_result.policy)
print(value_iteration_result.V)
print(value_iteration_result.iter)

Policy-Iteration Algorithm:
(0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 3, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 3, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 2, 1, 3, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 3, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 1,

In [34]:
def get_greedy_index(warehouse_state, action):
  if action.operation_type == OperationType.STORE:
    return warehouse_state.index(None)
  elif action.operation_type == OperationType.RESTORE:
    return warehouse_state.index(action.item_type)
  else:
    raise ValueError

In [35]:
def run_evaluation(dataset, states_space, policy):
    warehouse_state = [None] * WAREHOUSE_SIZE
    total_manhattan_distance = 0
    for action in dataset:
        position = -1
        if policy == 'greedy':
            position = get_greedy_index(warehouse_state, action)
        else:
            index = states_space.index((tuple(warehouse_state), action))
            position = policy[index]
        
        total_manhattan_distance += Warehouse.calculate_distance(position, WAREHOUSE_LAYOUT)
        warehouse_state = Warehouse.apply_at_to(action, position, warehouse_state)
    return total_manhattan_distance

## Evaluation on the training set

In [36]:
total_distance_policy_iteration = run_evaluation(train_dataset,
                                                 state_space,
                                                 policy_iteration_result.policy)

total_distance_value_iteration = run_evaluation(train_dataset,
                                                state_space,
                                                value_iteration_result.policy)

total_distance_greedy_selection = run_evaluation(train_dataset,
                                                state_space,
                                                'greedy')


print(total_distance_policy_iteration)
print(total_distance_value_iteration)
print(total_distance_greedy_selection)

14401
14401
14401


## Evaluation on the testset 

In [31]:
test_dataset = WarehouseDataSet(TEST_DATASET_PATH)

In [38]:
total_distance_policy_iteration = run_evaluation(test_dataset,
                                                 state_space,
                                                 policy_iteration_result.policy)

total_distance_value_iteration = run_evaluation(test_dataset,
                                                state_space,
                                                value_iteration_result.policy)
total_distance_greedy_selection = run_evaluation(test_dataset,
                                                state_space,
                                                'greedy')
print(total_distance_policy_iteration)
print(total_distance_value_iteration)
print(total_distance_greedy_selection)

114
114
114
